# Python Functions Review

### Introduction

In this lesson, we'll review working with Python APIs and functions.  Let's get started.

### Getting our Data

In this lesson, we'll work with the [Econdb api](https://www.econdb.com/api/series/?page=1).  As you can see on the page, we can make a request to the api with something like the following.

In [62]:
import requests

url = "https://www.econdb.com/api/series/CPIUS/?format=json"
response = requests.get(url)
json_values = response.json()

> If the above line did not work because you have not installed the requests library, you can install by uncommenting and running the following:

In [61]:
# !pip3 install requests

### Working with our data

Begin by identifying all of the keys in `json_values`.

In [63]:
json_values.keys()

# dict_keys(['ticker', 'description', 'geography', 
# 'frequency', 'dataset', 'units', 'additional_metadata', 'data'])

dict_keys(['ticker', 'description', 'geography', 'frequency', 'dataset', 'units', 'additional_metadata', 'data'])

And also identify all of the keys in under the `data` attribute.

In [3]:
json_values['data'].keys()

# dict_keys(['values', 'dates', 'status'])

dict_keys(['values', 'dates', 'status'])

Ok, so we can see that we have keys of `values` and `dates`.

So now create a list of the most recent 300 dates -- from earliest to latest.

In [65]:
recent_dates = json_values['data']['dates'][-300:]
recent_dates[:5]

# ['1997-08-01', '1997-09-01', '1997-10-01', '1997-11-01', '1997-12-01']

And create a list of the corresponding consumer price index values.

In [5]:
recent_values = json_values['data']['values'][-300:]
recent_values[:5]

[161.2, 161.5, 161.7, 161.8, 162.0]

Now create a list of tuples called `cpis` that uses the `recent_dates` and `recent_values` lists above to return a list of tuples consisting of the date and the corresponding consumer price index.

In [6]:
cpis = list(zip(recent_dates, recent_values))

cpis[:5]

# [('1997-08-01', 160.8),
#  ('1997-09-01', 161.2),
#  ('1997-10-01', 161.5),
#  ('1997-11-01', 161.7),
#  ('1997-12-01', 161.8)]

[('1997-09-01', 161.2),
 ('1997-10-01', 161.5),
 ('1997-11-01', 161.7),
 ('1997-12-01', 161.8),
 ('1998-01-01', 162.0)]

And then, turn this list of tuples into a list of dictionaries with keys of date and price.

> If you are unable to complete the below step, you can skip the problem and proceed to the next section.

In [9]:
price_json = [dict(zip(['date', 'price'], cpi))for cpi in cpis]

price_json[:5]

# [{'date': '1997-09-01', 'price': 161.2},
#  {'date': '1997-10-01', 'price': 161.5},
#  {'date': '1997-11-01', 'price': 161.7},
#  {'date': '1997-12-01', 'price': 161.8},
#  {'date': '1998-01-01', 'price': 162.0}]

### Working with Data

Ok, so in case you were not able to complete the probems above, we'll load in our list of dictionaries of prices and dates below.

In [66]:
import pandas as pd

price_dates = pd.read_json('./cpi_json.json').to_dict('records')

price_dates[:3]

[{'date': Timestamp('1997-09-01 00:00:00'), 'price': 161.2},
 {'date': Timestamp('1997-10-01 00:00:00'), 'price': 161.5},
 {'date': Timestamp('1997-11-01 00:00:00'), 'price': 161.7}]

Ok, so now write a function that given our list of dictionaries and a price, only returns those dictionaries whose price exceeds that amount.

* Filtering

In [32]:
def price_greater_than(price_dates, amount):
    return [price_date for price_date in price_dates if price_date['price'] > amount]

In [33]:
price_greater_than(price_dates, 200)[:3]

[{'date': Timestamp('2006-04-01 00:00:00'), 'price': 200.7},
 {'date': Timestamp('2006-05-01 00:00:00'), 'price': 201.3},
 {'date': Timestamp('2006-06-01 00:00:00'), 'price': 201.8}]

* Exploring Timestamps  

Now we can see that each date points to a timestamp object.  If we explore a timestamp, notice that we can extract certain attributes from it.

> Let's start by pulling out the first date.

In [36]:
first_price_date = price_dates[0]

first_price_date # {'date': Timestamp('1997-09-01 00:00:00'), 'price': 161.2}

first_date = first_price_date['date']
first_date

Timestamp('1997-09-01 00:00:00')

For example, this is how we find the price from a timestamp.

In [37]:
first_date.year

1997

Now, write a function called `filter_price_dates_by` that takes list of `price_dates` and given both a month and year, return the corresponding price.

> You can see how it works by looking at how we call the function below.

In [67]:
def filter_price_by(price_dates, month, year):
    prices = [price_date['price'] for price_date in price_dates if price_date['date'].month == month and price_date['date'].year == year] 
    if prices:
        return prices[0]

In [46]:
filter_price_by(price_dates, 3, 2020)

# 258.2

258.2

Then let's write a method that allows us to filter by both prices and years.  So provided a price and a year, it only returns the corresponding dictionaries for that year that exceeded the specified price.

In [47]:
def filter_price_by(amount, year):
    return [price_date for price_date in price_dates if (price_date['price'] > amount and price_date['date'].year > year)]

In [58]:
filter_price_by(290, 1970)

# [{'date': Timestamp('2022-05-01 00:00:00'), 'price': 291.5},
#  {'date': Timestamp('2022-06-01 00:00:00'), 'price': 295.3},
#  {'date': Timestamp('2022-07-01 00:00:00'), 'price': 295.3},
#  {'date': Timestamp('2022-08-01 00:00:00'), 'price': 295.6}]

[{'date': Timestamp('2022-05-01 00:00:00'), 'price': 291.5},
 {'date': Timestamp('2022-06-01 00:00:00'), 'price': 295.3},
 {'date': Timestamp('2022-07-01 00:00:00'), 'price': 295.3},
 {'date': Timestamp('2022-08-01 00:00:00'), 'price': 295.6}]

Finally,s create a new list of dictionaries so that each dictionary has a key of `year`, a key of `month`, as well as the corresponding price.

In [40]:
expanded_price_dates = [{'month': price_date['date'].month, 'year': price_date['date'].year, 'price': price_date['price']} for price_date in price_dates]

expanded_price_dates[:3]

# [{'month': 9, 'year': 1997, 'price': 161.2},
#  {'month': 10, 'year': 1997, 'price': 161.5},
#  {'month': 11, 'year': 1997, 'price': 161.7}]

[{'month': 9, 'year': 1997, 'price': 161.2},
 {'month': 10, 'year': 1997, 'price': 161.5},
 {'month': 11, 'year': 1997, 'price': 161.7}]